# [병원 개/폐업 분류 예측](https://dacon.io/competitions/official/9565/overview/)

[12/22] 여러 모델의 성능 평가

1. 앙상블 - voting [0.84251] : Logistic Regression + KNN + Decision Tree

- 다른 앙상블 모델 적용하기
- 모델 적용 후 전처리 연구해보기

In [7]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

In [2]:
!unzip drive/MyDrive/env/data02.zip

Archive:  drive/MyDrive/env/data02.zip
  inflating: train.csv               
  inflating: submission_sample.csv   
  inflating: test.csv                


In [14]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('submission_sample.csv')

In [15]:
pd.options.display.max_columns = None

In [16]:
train

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,open,choongnam,73,20071228,175.0,nursing_hospital,4.217530e+09,0.000000e+00,3.961135e+09,2.033835e+09,15652441.0,1.523624e+07,1.323624e+07,31641798.0,2.251697e+08,1.012700e+09,9.976719e+08,4.700557e+08,1.502781e+07,2.514586e+09,2.360684e+09,1.434496e+08,0.0,6.828260e+08,2.013237e+08,0.000000e+00,4.815023e+08,3.510000e+08,2.844460e+09,1.496394e+09,4.297848e+09,0.000000e+00,4.057422e+09,2.063787e+09,16194675.0,2.998335e+07,1.568305e+07,18808074.0,2.078297e+08,8.301695e+08,8.165705e+08,5.237026e+08,1.359897e+07,2.548115e+09,2.386263e+09,1.458986e+08,0.0,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,same
1,3,open,gyeongnam,32,19970401,410.0,general_hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,801.0,813.0,same
2,4,open,gyeonggi,89,20161228,468.0,nursing_hospital,1.004522e+09,5.154837e+08,4.472197e+08,2.964023e+08,76156.0,3.000000e+04,0.000000e+00,0.0,4.186475e+07,2.724421e+08,2.536822e+08,8.095950e+06,1.875997e+07,1.204810e+08,1.204810e+08,0.000000e+00,0.0,9.241434e+07,9.241434e+07,0.000000e+00,0.000000e+00,0.000000e+00,3.005088e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,same
3,7,open,incheon,141,20000814,353.0,general_hospital,7.250734e+10,0.000000e+00,7.067740e+10,3.178605e+10,506223059.0,1.259568e+09,1.196881e+09,173769780.0,9.028303e+08,1.304154e+10,1.153475e+10,0.000000e+00,1.506793e+09,4.317936e+10,3.832078e+10,3.945208e+09,0.0,4.236380e+10,2.799188e+10,1.988520e+10,1.437192e+10,7.253040e+09,1.385710e+10,8.643659e+09,6.685834e+10,0.000000e+00,6.492419e+10,2.971135e+10,476807804.0,1.353672e+09,1.277422e+09,218891720.0,8.383875e+08,1.112572e+10,9.890540e+09,0.000000e+00,1.235181e+09,3.958356e+10,3.485576e+10,3.915906e+09,0.0,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,same
4,9,open,gyeongnam,32,20050901,196.0,general_hospital,4.904354e+10,0.000000e+00,4.765605e+10,2.446078e+10,112352259.0,1.419089e+09,1.307249e+09,0.0,8.074970e+07,6.317084e+09,5.873265e+09,4.099320e+09,4.438186e+08,4.366733e+10,4.330613e+10,2.223400e+08,0.0,4.989811e+10,2.890919e+10,1.793038e+10,2.098892e+10,1.350000e+10,8.631164e+07,9.025550e+09,4.808280e+10,0.000000e+00,4.712580e+10,2.346004e+10,597748128.0,1.522108e+09,1.349851e+09,0.0,3.264258e+07,4.906776e+09,4.464017e+09,3.365227e+09,4.427591e+08,4.653138e+10,4.562945e+10,7.893407e+08,0.0,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,same
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,422,open,jeonnam,178,20050211,214.0,nursing_hospital,7.614697e+09,4.153475e+08,5.903119e+09,3.467861e+09,42098974.0,7.439421e+08,1.875761e+08,0.0,5.943868e+08,2.776603e+09,2.769516e+09,1.797116e+09,7.086391e+06,1.191266e+10,1.171868e+10,1.198567e+08,0.0,8.715369e+09,1.520004e+09,0.000

### 데이터 분포 확인

In [17]:
unique, counts = np.unique(train.OC, return_counts=True)
dict(zip(unique, counts))

{' close': 15, 'open': 286}

In [18]:
test.employee1= test.employee1.str.replace(",","").astype(np.float64)
test.employee2= test.employee2.str.replace(",","").astype(np.float64)

### 결측치 많이 발생하는 수로 대체

In [19]:
imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_ = imp_freq.fit_transform(train.iloc[:,2:])
test_ = imp_freq.fit_transform(test.iloc[:,2:])

print(train_.shape)
print(test_.shape)
print(test_)

(301, 56)
(127, 56)
[['incheon' 139 19981125.0 ... 693.0 693.0 'same']
 ['jeju' 149 20160309.0 ... 379.0 371.0 'same']
 ['jeonnam' 103 19890427.0 ... 44.0 42.0 'same']
 ...
 ['gyeonggi' 116 20010101.0 ... 44.0 42.0 'same']
 ['jeju' 76 20010201.0 ... 363.0 343.0 'same']
 ['choongbuk' 163 19961216.0 ... 44.0 42.0 'same']]


### 라벨링을 위해 train과 test 병합

In [20]:
total_df = np.concatenate((train_, test_), axis = 0)

print(total_df)
print(total_df.shape)

[['choongnam' 73 20071228 ... 62.0 64.0 'same']
 ['gyeongnam' 32 19970401 ... 801.0 813.0 'same']
 ['gyeonggi' 89 20161228 ... 234.0 1.0 'same']
 ...
 ['gyeonggi' 116 20010101.0 ... 44.0 42.0 'same']
 ['jeju' 76 20010201.0 ... 363.0 343.0 'same']
 ['choongbuk' 163 19961216.0 ... 44.0 42.0 'same']]
(428, 56)


In [21]:
le = preprocessing.LabelEncoder()

total_df[:,0] = le.fit_transform(total_df[:,0])
total_df[:,4] = le.fit_transform(total_df[:,4])
total_df[:,-1] = le.fit_transform(total_df[:,-1])

In [22]:
x_train = total_df[:301,:]
x_test = total_df[301:,:]

y_train = train.iloc[:,1]
y_train = le.fit_transform(y_train)

In [23]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(301, 56)
(301,)
(127, 56)


In [24]:
scaler = MinMaxScaler()
x_std_train = scaler.fit_transform(x_train)
x_std_test = scaler.transform(x_test)

### 앙상블 - voting 적용을 위해 개별 모델 및 앙상블 모델 적용

In [30]:
## 앙상블 모델 써보기
logistic = LogisticRegression(solver='liblinear',
                              penalty='l2',
                              C=0.001,
                              random_state=1)

tree = DecisionTreeClassifier(max_depth=None,
                              criterion='entropy',
                              random_state=1)

knn = KNeighborsClassifier(n_neighbors=1,
                            p=2,
                            weights = 'distance',
                            metric='minkowski')

vote_estimators = [('logistic', logistic), ('tree', tree), ('knn', knn)]
vote = VotingClassifier(estimators = vote_estimators,
                          voting='soft')

### K-fold 교차 검증을 통한 모델 평가

In [31]:
clf_labels = ['Logistic regression', 'Decision tree', 'KNN', 'Majority voting']
all_clf = [logistic, tree, knn, vote]

for clf, label in zip(all_clf, clf_labels):
    scores = cross_val_score(estimator=clf, X=x_std_train, y=y_train, cv=10, scoring='accuracy')
    print(label, ":", scores.mean())

Logistic regression : 0.9502150537634408
Decision tree : 0.9137634408602151
KNN : 0.9203225806451613
Majority voting : 0.9502150537634408


-> 검증에서는 앙상블 모델의 성능이 가장 좋음

### GridSearch 방식을 통해 하이퍼 파라미터 튜닝

In [32]:
params = {'logistic__C': [0.0001, 0.001, 0.01, 1],
          'tree__max_depth': [1, 3, 5],
          'knn__n_neighbors': [1, 3, 5]}

vote_grid = GridSearchCV(estimator=vote,
                    param_grid=params,
                    cv=10,
                    scoring='accuracy',
                    iid=False)

vote_grid.fit(x_std_train, y_train)
    
print('최적의 파타미터: %s' % vote_grid.best_params_)
print('ACU: %f' % vote_grid.best_score_)    

최적의 파타미터: {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 3}
ACU: 0.950215


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [33]:
predict = vote_grid.predict(x_std_train)
y_pred = vote_grid.predict(x_std_test)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [34]:
submit['OC'] = y_pred.astype(np.int)
submit.to_csv('submission.csv', index = False)